# 1. 개발 환경 설정

# 개발 환경: 코랩 A100

### 1.1 필수 라이브러리 설치하기

In [ ]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.0 

In [ ]:
# prompt: i want to print out the versions only

!nvidia-smi
!echo "bitsandbytes version:"
!pip show bitsandbytes
!echo "peft version:"
!pip show peft
!echo "trl version:"
!pip show trl
!echo "accelerate version:"
!pip show accelerate
!echo "datasets version:"
!pip show datasets
!echo "transformers version:"
!pip show transformers

Wed Aug 14 12:16:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

Mounted at /content/drive


### 1.2 Import modules

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm
from datasets import DatasetDict
from accelerate import Accelerator

# 2. Dataset 생성 및 준비

### 2.0 csv파일 불러오기

In [ ]:
# open.zip 파일을 업로드한 경로를 확인하여 적용
!unzip  "/content/drive/MyDrive/2024인하대/open.zip"

Archive:  /content/drive/MyDrive/2024인하대/open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
file_path = '/content/test.csv'
test_data = pd.read_csv(file_path)

In [ ]:
dataset = Dataset.from_pandas(test_data)
dataset

Dataset({
    features: ['id', 'context', 'question'],
    num_rows: 1507
})

In [ ]:
dataset[0]

{'id': 'TEST_0000',
 'context': '신종 코로나바이러스 감염증(코로나19) 확산 속에 제주특별자치도 경제통상진흥원(원장 문관영)이 추진하는 해외 시장 진출 지원 사업이 지역 중소기업과 소상공인들에게 큰 힘이 되고 있다. \n\n제주경제통상진흥원은 우수한 제품을 생산하고도 판로 개척에 어려움을 겪는 도내 유망 중소기업을 위해 해외통상사무소(상해대표처, 동경통상대표부)를 통해 중국과 일본을 중심으로 온라인 판매, 비대면 마케팅 지원 사업을 꾸준히 펼치고 있다. \n\n제주경제통상진흥원 해외통상사무소는 신규 바이어 발굴, 지역 업체와 바이어 간 매칭 및 미팅 주선, 제주 상품 통관 및 수출 관련 절차 지원, 수출 성사 및 실적 관리 등 제주지역 수출업체의 해외지사 역할을 수행하면서 호응을 얻고 있다. \n\n‘알리바바’ 등 현지 온라인몰에서 제주관을 운영하고 대형 쇼핑몰에 제주 상품 전용매대를 설치ㆍ운영하고 인플우언서 마케팅, 온ㆍ오프라인 판촉 이벤트도 활발히 펼치면서 제주 기업의 자생력을 키우고 있다. \n\n경제통상진흥원은 현지 대형 쇼핑몰에 제주 상품 전용 매대를 설치ㆍ운영하며 제주를 홍보하는 데도 앞장 서고 있다. \n\n중국 상해대표처=중국 상하이시 장녕구에 있는 상해대표처(소장 정명구)는 2015년 8월 설립 이후 제주 상품의 중국 진출과 기업 활동 지원 업무를 맡고 있다. \n\n지난해에는 ‘위챗’을 통해 40개사 285품목, ‘왕홍’ 생방송을 통해 3개사 5품목을 홍보하고 광명마트, 세븐일레븐, 올레마트 등 오프라인 178개 매장에서 7개사 16품목을 입점 지원했다. \n\n통상, 투자유치, 교류, 관광사업 지원 등 공적 기능을 수행하는 상해대표처는 지난해 제주지역 업체에 대한 중화권 수출(29개사 17억8800만원)에 대한 업무 지원(통관, 물류, 샘플 발송, 현지 판촉행사 등)을 수행했다. \n\n이보다 앞선 2019년에도 현지 바이어 방문을 통한 제주 제품 소개로 38건(17억원)에 대한 수출을 성사시켰고 온ㆍ오프라인 마케

# 3. Gemma 모델 추론

### 3.1 Fine-tuned 모델 로드

In [ ]:
base_model = "rtzr/ko-gemma-2-9b-it"

### Load basemodel
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.float16,
    device_map= {"": 0}
)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
# 저장한 어댑터 모델 불러오기
newModelPath = '/content/gemma2_lora_adapter'

In [ ]:
# 베이스 모델 및 어댑터 연결
connectedModel = PeftModel.from_pretrained(baseModel, newModelPath)

In [ ]:
#모델 병합 (메모리 터짐) ColabPro에서만 활용
mergedModel = connectedModel.merge_and_unload()

### 3.2 Fine-tuned 모델 추론

In [ ]:
pipe_finetuned = pipeline("text-generation", model=mergedModel, tokenizer=tokenizer, max_new_tokens=100)

In [ ]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    outputs = pipe_finetuned(
    prompt,
    do_sample=False,
    add_special_tokens=True
)
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
submission_dict = {}
count = 0

for i in range(len(dataset)):
    try:
        context = dataset['context'][i]
        question = dataset['question'][i]
        id = dataset['id'][i]


        if context is not None and question is not None:
            messages = [
                {
                    "role": "user",
                    "content": "너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.:\n\n{} \n\n\
                    Question:\n{}".format(context, question)
                }
            ]
            prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            answer = generate_response(prompt)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

        print("Answer for question:", question, ":", submission_dict[id])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")

Answer for question: 어떤 기관이 지역 중소기업과 소상공인들에게 큰 힘이 되는 통상진흥원인가요 : 제주특별자치도 경제통상진흥원
Processed count: 1
Answer for question: 제주 경제통상진흥원이 어떤 해외 시장을 우선적으로 공략하고 있나요 : 중국과 일본
Processed count: 2
Answer for question: 상해대표처는 어느 시점에 설립되었습니까 : 2015년 8월
Processed count: 3
Answer for question: 동경통상대표부는 어느 지역에 설립되었습니까 : 도쿄
Processed count: 4
Answer for question: 제주경제통상진흥원이 지난해에 도내에서 생산된 제품에 대한 일본 수출에 얼마를 지원했나요 : 31억7200만원
Processed count: 5
Answer for question: 서경대 실용음악과 보컬전공의 대입 시험은 어디에서 진행되었나 : 서경대 풋살경기장
Processed count: 6
Answer for question: 천막 고사장으로 변경한 이유는 무엇인가 : 코로나19
Processed count: 7
Answer for question: 2021학년도 서경대 실용음악과 보컬 전공 수시에 몇 명이 지원했나 : 1710명
Processed count: 8
Answer for question: 하나의 시험 당 약 몇 명이 참가했나 : 25명
Processed count: 9
Answer for question: 기존과 같은 방식으로 대면 면접·실기시험을 치르는 학교들은 어떤 방식을 채택했나 : 거리두기
Processed count: 10
Answer for question: 중소기업 혁신바우처 사업이란 어떤 사업인가 : 제조 중소기업의 경쟁력 제고를 위한
Processed count: 11
Answer for question: 이번에 신설된 컨설팅 서비스 프로그램이 무엇인가 : 탄소중립 경영혁신
Processed count: 12
Ans

In [ ]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( '/content/gemma2_submission.csv', index=False)

In [ ]:
del mergedModel
del connectedModel
del baseModel
del pipe_finetuned
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

411

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# 4. eeve 모델 추론

### 4.1 Fine-tuned 모델 로드

In [ ]:
base_model2 = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

### Load basemodel
baseModel2 = AutoModelForCausalLM.from_pretrained(
    base_model2,
    low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.float16,
    device_map= {"": 0}
)

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained(base_model2)

tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [ ]:
# 저장한 어댑터 모델 불러오기
newModelPath = '/content/eeve_lora_adapter'

In [ ]:
# 베이스 모델 및 어댑터 연결
connectedModel = PeftModel.from_pretrained(baseModel2, newModelPath)

In [ ]:
#모델 병합 (메모리 터짐) ColabPro에서만 활용
mergedModel = connectedModel.merge_and_unload()

### 4.2 Fine-tuned 모델 추론

In [ ]:
pipe_finetuned = pipeline("text-generation", model=mergedModel, tokenizer=tokenizer2, max_new_tokens=100)

In [ ]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    outputs = pipe_finetuned(
    prompt,
    do_sample=False,
    add_special_tokens=True
)
    response = outputs[0]["generated_text"][len(prompt):]
    # 토큰 반복 생성 및 노이즈 토큰 관련 처리
    if "Que" in response:
      response = response.split("Que", 1)[0]
    if "⊙" in response:
      response = response.split("⊙", 1)[0]
    if "Con" in response:
      response = response.split("Con", 1)[0]
    if "\n" in response:
      response = response.split("\n", 1)[0]
    if "<eos>" in response:
      response = response.split("<eos>", 1)[0]
    if "assistant" in response:
      response = response.split("assistant", 1)[0]

    return response

In [ ]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
submission_dict = {}
count = 0

for i in range(len(dataset)):
    try:
        context = dataset['context'][i]
        question = dataset['question'][i]
        id = dataset['id'][i]


        if context is not None and question is not None:
            messages = [
                {
                    "role": "user",
                    "content": "너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.:\n\n{} \n\n\
                    Question:\n{}".format(context, question)
                }
            ]
            prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            answer = generate_response(prompt)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

        print("Answer for question:", question, ":", submission_dict[id])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")

Answer for question: 어떤 기관이 지역 중소기업과 소상공인들에게 큰 힘이 되는 통상진흥원인가요 : 제주특별자치도 경제통상진흥원
Processed count: 1
Answer for question: 제주 경제통상진흥원이 어떤 해외 시장을 우선적으로 공략하고 있나요 : 중국과 일본
Processed count: 2
Answer for question: 상해대표처는 어느 시점에 설립되었습니까 : 2015년 8월
Processed count: 3
Answer for question: 동경통상대표부는 어느 지역에 설립되었습니까 : 도쿄 신주쿠
Processed count: 4
Answer for question: 제주경제통상진흥원이 지난해에 도내에서 생산된 제품에 대한 일본 수출에 얼마를 지원했나요 : 31억7200만원
Processed count: 5
Answer for question: 서경대 실용음악과 보컬전공의 대입 시험은 어디에서 진행되었나 : 풋살경기장
Processed count: 6
Answer for question: 천막 고사장으로 변경한 이유는 무엇인가 : 코로나19
Processed count: 7
Answer for question: 2021학년도 서경대 실용음악과 보컬 전공 수시에 몇 명이 지원했나 : 1710명
Processed count: 8
Answer for question: 하나의 시험 당 약 몇 명이 참가했나 : 25명
Processed count: 9
Answer for question: 기존과 같은 방식으로 대면 면접·실기시험을 치르는 학교들은 어떤 방식을 채택했나 : 야외
Processed count: 10
Answer for question: 중소기업 혁신바우처 사업이란 어떤 사업인가 : 제조 중소기업의 경영혁신과 원활한 회생과 재기 활동을 지원하기 위해 컨설팅, 기술지원, 마케팅 서비스를 제공하는 사업
Processed count: 11
Answer for question: 이번에 신설된 컨설팅 서비스 프로

In [ ]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( '/content/eeve_submission.csv', index=False)

In [ ]:
del baseModel2
del mergedModel
del connectedModel
del pipe_finetuned
torch.cuda.empty_cache()

In [ ]:
gc.collect()

108

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# 5. nous 모델 추론

In [ ]:
# 모델 머지 후 저장 (향후 베이스 모델없이 단독으로 사용가능) T4 GPU 시 OutOfMemory 가능 Colab Pro
base_model = "NousResearch/Nous-Hermes-2-SOLAR-10.7B"

### Load basemodel
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.float16,
    device_map= {"": 0}
)

model-00002-of-00005.safetensors:  37%|###7      | 1.86G/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
### Load basemodel's tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

In [ ]:
# 저장한 어댑터 모델 불러오기
newModelPath = '/content/nous_solar_training_final'

In [ ]:
# 베이스 모델 및 어댑터 연결
connectedModel = PeftModel.from_pretrained(baseModel, newModelPath)

In [ ]:
#모델 병합 (메모리 터짐) ColabPro에서만 활용
mergedModel = connectedModel.merge_and_unload()

In [ ]:
pipe = pipeline(task="text-generation", model=mergedModel, tokenizer=tokenizer)

def extract_response_solar(input, prompt):
    result = pipe(f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                    Question에 대한 답변만 최대한 한 단어로, 명사들로만 답변해줘. \
                    예를 들면 누구(사람에대한 질문)-> 김철수(사람 이름만, 직무 회사 제외), 물건 -> 사과, 시간 -> 1시, 속력 -> 1m/s, 이런식으로 딱 간단하게만 나타내줘. \
                    ### System:\n{input}\n\n### User:\n{prompt}\n\n### Assistant:\n",
                do_sample = False,  # 샘플링 방법을 지정. True : 모델이 확률에 따라 텍스트를 샘플링
                max_new_tokens=100,
                num_return_sequences = 1,  # 생성할 텍스트 시퀀스의 수를 지정
                pad_token_id=tokenizer.eos_token_id
)
    answer=result[0]['generated_text'].split("Assistant:\n")[-1]
     # 토큰 반복 생성 및 노이즈 토큰 관련 처리
    if "Que" in answer:
        answer = answer.split("Que", 1)[0]
    if "⊙" in answer:
        answer = answer.split("⊙", 1)[0]
    if "Con" in answer:
        answer = answer.split("Con", 1)[0]
    if "\n" in answer:
        answer = answer.split("\n", 1)[0]
    if "  " in answer:
        answer = answer.split("  ", 1)[0]

    return answer

In [ ]:
predict_dict = {}
count = 0

for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        id= row['id']


        if context is not None and question is not None:

            answer = extract_response_solar(context,question)
            predict_dict[id] = answer
        else:
            predict_dict[id] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[id])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")

Answer for question: 어떤 기관이 지역 중소기업과 소상공인들에게 큰 힘이 되는 통상진흥원인가요 : 제주경제통상진흥원
Processed count: 1
Answer for question: 제주 경제통상진흥원이 어떤 해외 시장을 우선적으로 공략하고 있나요 : 중국과 일본
Processed count: 2
Answer for question: 상해대표처는 어느 시점에 설립되었습니까 : 2015년 8월
Processed count: 3
Answer for question: 동경통상대표부는 어느 지역에 설립되었습니까 : 도쿄
Processed count: 4
Answer for question: 제주경제통상진흥원이 지난해에 도내에서 생산된 제품에 대한 일본 수출에 얼마를 지원했나요 : 31억7200만원
Processed count: 5
Answer for question: 서경대 실용음악과 보컬전공의 대입 시험은 어디에서 진행되었나 : 운동장
Processed count: 6
Answer for question: 천막 고사장으로 변경한 이유는 무엇인가 : 코로나19
Processed count: 7
Answer for question: 2021학년도 서경대 실용음악과 보컬 전공 수시에 몇 명이 지원했나 : 1710명
Processed count: 8
Answer for question: 하나의 시험 당 약 몇 명이 참가했나 : 1710명
Processed count: 9
Answer for question: 기존과 같은 방식으로 대면 면접·실기시험을 치르는 학교들은 어떤 방식을 채택했나 : 야외나 대규모 시설
Processed count: 10
Answer for question: 중소기업 혁신바우처 사업이란 어떤 사업인가 : 제조 중소기업의 경영혁신과 원활한 회생과 재기 활동을 지원하기 위해 컨설팅, 기술지원, 마케팅 서비스를 제공하는 사업
Processed count: 11
Answer for question: 이번에 신설된 컨설팅 서비스 프로그램

In [ ]:
df = pd.DataFrame(list(predict_dict.items()), columns=['id', 'answer'])
df.head()

,id,answer
0,TEST_0000,제주경제통상진흥원
1,TEST_0001,중국과 일본
2,TEST_0002,2015년 8월
3,TEST_0003,도쿄
4,TEST_0004,31억7200만원


In [ ]:
df.to_csv('/content/nous_submission.csv',index=False)

In [ ]:
del baseModel
del mergedModel
del connectedModel
del pipe
torch.cuda.empty_cache()

In [ ]:
gc.collect()

52

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# 6. 앙상블

In [ ]:
file_path = '/content/gemma2_submission.csv'
test_inf1 = pd.read_csv(file_path)

In [ ]:
file_path = '/content/eeve_submission.csv'
test_inf2 = pd.read_csv(file_path)

In [ ]:
file_path = '/content/nous_submission.csv'
test_inf3 = pd.read_csv(file_path)

In [ ]:
test_inf1.head()

,id,answer
0,TEST_0000,제주특별자치도 경제통상진흥원
1,TEST_0001,중국과 일본
2,TEST_0002,2015년 8월
3,TEST_0003,도쿄
4,TEST_0004,31억7200만원


In [ ]:
test_inf2.head()

,id,answer
0,TEST_0000,제주특별자치도 경제통상진흥원
1,TEST_0001,중국과 일본
2,TEST_0002,2015년 8월
3,TEST_0003,도쿄 신주쿠
4,TEST_0004,31억7200만원


In [ ]:
test_inf3.head()

,id,answer
0,TEST_0000,제주경제통상진흥원
1,TEST_0001,중국과 일본
2,TEST_0002,2015년 8월
3,TEST_0003,도쿄
4,TEST_0004,31억7200만원


In [ ]:
# 'id'와 'answer' 열을 추출하여 새로운 데이터프레임 생성
ensemble = pd.DataFrame({
    'id': test_inf1['id'],
    'answer1': test_inf1['answer'],
    'answer2': test_inf2['answer'],
    'answer3': test_inf3['answer']
})

In [ ]:
ensemble.head()

,id,answer1,answer2,answer3
0,TEST_0000,제주특별자치도 경제통상진흥원,제주특별자치도 경제통상진흥원,제주경제통상진흥원
1,TEST_0001,중국과 일본,중국과 일본,중국과 일본
2,TEST_0002,2015년 8월,2015년 8월,2015년 8월
3,TEST_0003,도쿄,도쿄 신주쿠,도쿄
4,TEST_0004,31억7200만원,31억7200만원,31억7200만원


### Define F1 score

In [ ]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [ ]:
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

# F1 스코어 계산 및 결과 추가 함수
def add_f1_scores(df):
    df['f1_answer1_vs_answer2'] = df.apply(lambda row: f1_score(row['answer1'], row['answer2']), axis=1)
    df['f1_answer2_vs_answer3'] = df.apply(lambda row: f1_score(row['answer2'], row['answer3']), axis=1)
    df['f1_answer1_vs_answer3'] = df.apply(lambda row: f1_score(row['answer1'], row['answer3']), axis=1)
    return df

# F1 스코어 계산 후 데이터프레임에 추가
ensemble_with_f1 = add_f1_scores(ensemble)
ensemble_with_f1

,id,answer1,answer2,answer3,f1_answer1_vs_answer2,f1_answer2_vs_answer3,f1_answer1_vs_answer3
0,TEST_0000,제주특별자치도 경제통상진흥원,제주특별자치도 경제통상진흥원,제주경제통상진흥원,1.000000,0.782609,0.782609
1,TEST_0001,중국과 일본,중국과 일본,중국과 일본,1.000000,1.000000,1.000000
2,TEST_0002,2015년 8월,2015년 8월,2015년 8월,1.000000,1.000000,1.000000
3,TEST_0003,도쿄,도쿄 신주쿠,도쿄,0.571429,0.571429,1.000000
4,TEST_0004,31억7200만원,31억7200만원,31억7200만원,1.000000,1.000000,1.000000
5,TEST_0005,서경대 풋살경기장,풋살경기장,운동장,0.769231,0.250000,0.181818
6,TEST_0006,코로나19,코로나19,코로나19,1.000000,1.000000,1.000000
7,TEST_0007,1710명,1710명,1710명,1.000000,1.000000,1.000000
8,TEST_0008,25명,25명,1710명,1.000000,0.250000,0.250000
9,TEST_0009,거리두기,야외,야외나 대규모 시설,0.000000,0.400000,0.000000


In [ ]:
ensemble_with_f1

,id,answer1,answer2,answer3,f1_answer1_vs_answer2,f1_answer2_vs_answer3,f1_answer1_vs_answer3
0,TEST_0000,제주특별자치도 경제통상진흥원,제주특별자치도 경제통상진흥원,제주경제통상진흥원,1.000000,0.782609,0.782609
1,TEST_0001,중국과 일본,중국과 일본,중국과 일본,1.000000,1.000000,1.000000
2,TEST_0002,2015년 8월,2015년 8월,2015년 8월,1.000000,1.000000,1.000000
3,TEST_0003,도쿄,도쿄 신주쿠,도쿄,0.571429,0.571429,1.000000
4,TEST_0004,31억7200만원,31억7200만원,31억7200만원,1.000000,1.000000,1.000000
5,TEST_0005,서경대 풋살경기장,풋살경기장,운동장,0.769231,0.250000,0.181818
6,TEST_0006,코로나19,코로나19,코로나19,1.000000,1.000000,1.000000
7,TEST_0007,1710명,1710명,1710명,1.000000,1.000000,1.000000
8,TEST_0008,25명,25명,1710명,1.000000,0.250000,0.250000
9,TEST_0009,거리두기,야외,야외나 대규모 시설,0.000000,0.400000,0.000000


In [ ]:
# F1 스코어 계산 후 데이터프레임에 추가
ensemble = add_f1_scores(ensemble)

# 새로운 'answer' 컬럼 생성 함수
def determine_answer(row):
    answers = [row['answer1'], row['answer2'], row['answer3']]
    # 각 값의 빈도를 세기
    answer_counts = pd.Series(answers).value_counts()
    # 가장 많이 나온 값을 선택 (빈도수가 같다면 첫 번째 값을 선택)
    most_common = answer_counts.idxmax()
    # 만약 모든 값이 다르면 f1 score 합이 가장 높은 값을 선택
    if answer_counts.iloc[0] == 1:
        f1_sum_answer1 = row['f1_answer1_vs_answer2'] + row['f1_answer1_vs_answer3']
        f1_sum_answer2 = row['f1_answer1_vs_answer2'] + row['f1_answer2_vs_answer3']
        f1_sum_answer3 = row['f1_answer2_vs_answer3'] + row['f1_answer1_vs_answer3']

        f1_sums = {
            row['answer1']: f1_sum_answer1,
            row['answer2']: f1_sum_answer2,
            row['answer3']: f1_sum_answer3
        }

        return max(f1_sums, key=f1_sums.get)
    return most_common

# 새로운 'answer' 컬럼을 데이터프레임에 추가
ensemble['answer'] = ensemble.apply(determine_answer, axis=1)

ensemble

,id,answer1,answer2,answer3,f1_answer1_vs_answer2,f1_answer2_vs_answer3,f1_answer1_vs_answer3,answer
0,TEST_0000,제주특별자치도 경제통상진흥원,제주특별자치도 경제통상진흥원,제주경제통상진흥원,1.000000,0.782609,0.782609,제주특별자치도 경제통상진흥원
1,TEST_0001,중국과 일본,중국과 일본,중국과 일본,1.000000,1.000000,1.000000,중국과 일본
2,TEST_0002,2015년 8월,2015년 8월,2015년 8월,1.000000,1.000000,1.000000,2015년 8월
3,TEST_0003,도쿄,도쿄 신주쿠,도쿄,0.571429,0.571429,1.000000,도쿄
4,TEST_0004,31억7200만원,31억7200만원,31억7200만원,1.000000,1.000000,1.000000,31억7200만원
5,TEST_0005,서경대 풋살경기장,풋살경기장,운동장,0.769231,0.250000,0.181818,풋살경기장
6,TEST_0006,코로나19,코로나19,코로나19,1.000000,1.000000,1.000000,코로나19
7,TEST_0007,1710명,1710명,1710명,1.000000,1.000000,1.000000,1710명
8,TEST_0008,25명,25명,1710명,1.000000,0.250000,0.250000,25명
9,TEST_0009,거리두기,야외,야외나 대규모 시설,0.000000,0.400000,0.000000,야외


In [ ]:
ensemble.tail(50)

,id,answer1,answer2,answer3,f1_answer1_vs_answer2,f1_answer2_vs_answer3,f1_answer1_vs_answer3,answer
0,TEST_0000,제주특별자치도 경제통상진흥원,제주특별자치도 경제통상진흥원,제주경제통상진흥원,1.000000,0.782609,0.782609,제주특별자치도 경제통상진흥원
1,TEST_0001,중국과 일본,중국과 일본,중국과 일본,1.000000,1.000000,1.000000,중국과 일본
2,TEST_0002,2015년 8월,2015년 8월,2015년 8월,1.000000,1.000000,1.000000,2015년 8월
3,TEST_0003,도쿄,도쿄 신주쿠,도쿄,0.571429,0.571429,1.000000,도쿄
4,TEST_0004,31억7200만원,31억7200만원,31억7200만원,1.000000,1.000000,1.000000,31억7200만원
5,TEST_0005,서경대 풋살경기장,풋살경기장,운동장,0.769231,0.250000,0.181818,풋살경기장
6,TEST_0006,코로나19,코로나19,코로나19,1.000000,1.000000,1.000000,코로나19
7,TEST_0007,1710명,1710명,1710명,1.000000,1.000000,1.000000,1710명
8,TEST_0008,25명,25명,1710명,1.000000,0.250000,0.250000,25명
9,TEST_0009,거리두기,야외,야외나 대규모 시설,0.000000,0.400000,0.000000,야외


In [ ]:
# id와 answer 열만 선택하여 CSV 파일로 저장
ensemble[['id', 'answer']].to_csv('/content/ensemble_final.csv', index=False)